## Get and Upload Complete Data to MariaDB and AWS S3
From the official nintendo website the following fields were obtained:
- Image
- Release Date
- Description

In [1]:
import os
import json
import requests
from time import sleep

# Database
import mysql.connector as mariadb

# Webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# AWS S3
import boto3
from io import BytesIO

In [3]:
# Upload switch games to database:
#  - id
#  - game_title


def add_switch_game(id, game_title, cursor):
    statement = 'INSERT INTO switch_games (id, game_title) VALUES (%s, %s)'
    data = (id, game_title)
    cursor.execute(statement, data)


def upload_switch_games():
    json_file = open('../dataset/switch-games-titles.json')
    switch_games = json.load(json_file)

    connection = mariadb.connect(
        user='root',
        password=os.environ.get('PROJECTS_PERSONAL_KEY'),
        host='34.136.74.202',
        database='games'
    )
    cursor = connection.cursor()

    for game in switch_games:
        add_switch_game(game['id'], game['title'], cursor)
    
    connection.commit()

upload_switch_games()

In [ ]:
def upload_image_to_s3(s3_client, file_name, image_url):
    bucket = 'nyzyss-the-eternal'
    res = requests.get(image_url, stream = True)

    buffer = BytesIO(res.content)
    
    s3_client.upload_fileobj(
        buffer,
        bucket,
        file_name,
        ExtraArgs={"ContentType": "image/png"},
    )


def get_extra_information(games_list):
    """
        Nintedo official page was used to obtain extra information and image of the game
    """
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://www.nintendo.com/')
    s3_client = boto3.client('s3')
    for game in games_list:
        driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[1]/div[1]/form/button').click()
        driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[1]/div[1]/div/div[1]/div[1]/form/div[1]/div/input').send_keys(game[1])
        driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[1]/div[1]/div/div[1]/div[1]/form/div[1]/div/input').send_keys(Keys.RETURN)
        sleep(1.5)
        driver.find_element(By.XPATH, '//*[@id="all-tab"]/section[1]/div[2]/div/div[1]/div/div[1]/div/div/div[1]/a').click()

        # try age verification
        try:
            driver.find_element(By.XPATH, '//*[@id="MM-Month"]').send_keys('01011990')
            driver.find_element(By.XPATH, '//*[@id="main"]/div[2]/div/form/button').click()
        except:
            pass

        # Get data
        game_image = driver.find_element(By.XPATH, '//*[@id="main"]/section[1]/div/div[2]/div[1]/div/section/div[2]/div/div[1]/div/div/div/div/img').get_attribute('src')
        try:
            release_date = driver.find_element(By.XPATH, '//*[@id="about"]/div/div[2]/div/div[1]/div').text
        except:
            release_date = driver.find_element(By.XPATH, '//*[@id="about"]/div/div/div/div[1]/div').text
        game_description = driver.find_element(By.XPATH, '//*[@id="main"]/section[2]/div/div/div[1]/div/div').text.replace(',', ' ').replace("'", "").replace('"', '')
        
        # Upload Image to S3 service
        upload_image_to_s3(s3_client, game[0], game_image)
        image_aws_url = 'https://nyzyss-the-eternal.s3.amazonaws.com/{0}.png'.format(game[0])

        # Upload url image and release_date to DB
        connection = mariadb.connect(
            user='root',
            password=os.environ.get('PROJECTS_PERSONAL_KEY'),
            host='34.172.27.136',
            database='games'
        )
        cursor = connection.cursor()
        query = '''
            UPDATE switch_games
            SET release_date = '{0}',
                image = '{1}',
                game_description = '{2}'
            WHERE id = '{3}'
        '''.format(release_date, image_aws_url, game_description, game[0])
        cursor.execute(query)
        connection.commit()
    driver.close()


def search_n_upload_data():
    connection = mariadb.connect(
        user='root',
        password=os.environ.get('PROJECTS_PERSONAL_KEY'),
        host='34.172.27.136',
        database='games'
    )
    cursor = connection.cursor()
    query = '''
        SELECT id, game_title
        FROM switch_games
        WHERE game_description IS NULL
    '''
    cursor.execute(query)
    games_list = cursor.fetchall()
    connection.close()

    get_extra_information(games_list)